# Real-World K-Means Applications

In this notebook, we'll apply K-Means to two practical problems:

1. **Customer Segmentation**: Group mall customers by shopping behavior
2. **Image Compression**: Reduce image file size using color clustering

For each example, we'll:
- Perform exploratory data analysis (EDA)
- Apply K-Means clustering
- Interpret and visualize results
- Extract business insights and recommendations

In [ ]:
# Import libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.graph_objects as go
import plotly.express as px
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
from sklearn.decomposition import PCA
from PIL import Image
import sys
sys.path.append('../src')

from utils import (
    load_mall_customers,
    load_sample_image,
    plot_clusters_3d,
    plot_elbow,
    plot_silhouette_scores,
    reduce_dimensions_pca
)

import warnings
warnings.filterwarnings('ignore')

plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette('Set2')

print("✅ Libraries imported successfully!")

---

# Example 1: Customer Segmentation

## Business Context

A shopping mall wants to understand their customers better to:
- **Target marketing campaigns** to specific customer groups
- **Optimize product placement** based on customer types
- **Personalize offers** to increase sales

We have data on customers' age, income, and spending behavior.

## Goal
Identify distinct customer segments using K-Means clustering.

## 1.1 Load and Explore Data

In [ ]:
# Load customer data
df = load_mall_customers()

print("Dataset shape:", df.shape)
print("\nFirst few rows:")
print(df.head(10))

In [ ]:
# Basic statistics
print("\n📊 Dataset Statistics:\n")
print(df.describe())

print("\n🔍 Data Types:")
print(df.dtypes)

print("\n❓ Missing Values:")
print(df.isnull().sum())

In [ ]:
# Gender distribution
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

# Gender pie chart
gender_counts = df['Gender'].value_counts()
axes[0].pie(gender_counts, labels=gender_counts.index, autopct='%1.1f%%', startangle=90, colors=['#3498db', '#e74c3c'])
axes[0].set_title('Gender Distribution', fontweight='bold')

# Age distribution
axes[1].hist(df['Age'], bins=20, color='#2ecc71', alpha=0.7, edgecolor='black')
axes[1].set_xlabel('Age')
axes[1].set_ylabel('Frequency')
axes[1].set_title('Age Distribution', fontweight='bold')
axes[1].grid(alpha=0.3)

# Income distribution
axes[2].hist(df['Annual Income (k$)'], bins=20, color='#f39c12', alpha=0.7, edgecolor='black')
axes[2].set_xlabel('Annual Income (k$)')
axes[2].set_ylabel('Frequency')
axes[2].set_title('Income Distribution', fontweight='bold')
axes[2].grid(alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
# Spending score distribution
plt.figure(figsize=(10, 6))
plt.hist(df['Spending Score (1-100)'], bins=20, color='#9b59b6', alpha=0.7, edgecolor='black')
plt.xlabel('Spending Score (1-100)', fontsize=12)
plt.ylabel('Frequency', fontsize=12)
plt.title('Spending Score Distribution', fontsize=14, fontweight='bold')
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

print(f"Average spending score: {df['Spending Score (1-100)'].mean():.1f}")
print(f"Median spending score: {df['Spending Score (1-100)'].median():.1f}")

In [ ]:
# Correlation analysis
correlation_features = ['Age', 'Annual Income (k$)', 'Spending Score (1-100)']
corr_matrix = df[correlation_features].corr()

plt.figure(figsize=(8, 6))
sns.heatmap(corr_matrix, annot=True, cmap='coolwarm', center=0, 
            square=True, linewidths=1, cbar_kws={"shrink": 0.8})
plt.title('Feature Correlation Matrix', fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

print("\n💡 Insight: Low correlations suggest features capture different aspects of customer behavior.")

## 1.2 Feature Engineering and Preprocessing

In [ ]:
# Select features for clustering
# We'll use Age, Income, and Spending Score
features = ['Age', 'Annual Income (k$)', 'Spending Score (1-100)']
X = df[features].values

print(f"Feature matrix shape: {X.shape}")
print(f"Features: {features}")

# Standardize features (important for K-Means!)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

print("\n✅ Features standardized (mean=0, std=1)")
print(f"Scaled data shape: {X_scaled.shape}")

## 1.3 Find Optimal Number of Clusters

### Elbow Method

In [ ]:
# Elbow method
K_range = range(1, 11)
inertias = []

print("Running elbow analysis...")
for k in K_range:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    kmeans.fit(X_scaled)
    inertias.append(kmeans.inertia_)
    print(f"  K={k}: inertia={kmeans.inertia_:.2f}")

# Plot
fig = plot_elbow(K_range, inertias, title="Elbow Method: Customer Segmentation")
fig.show()

print("\n📍 The elbow appears around K=4 or K=5")

### Silhouette Analysis

In [ ]:
# Silhouette scores
K_range_sil = range(2, 11)
silhouette_scores = []

print("Running silhouette analysis...")
for k in K_range_sil:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    labels = kmeans.fit_predict(X_scaled)
    score = silhouette_score(X_scaled, labels)
    silhouette_scores.append(score)
    print(f"  K={k}: silhouette score={score:.3f}")

# Plot
fig = plot_silhouette_scores(K_range_sil, silhouette_scores, 
                              title="Silhouette Analysis: Customer Segmentation")
fig.show()

best_k = list(K_range_sil)[np.argmax(silhouette_scores)]
print(f"\n🏆 Best K by silhouette score: {best_k}")
print(f"Silhouette score: {max(silhouette_scores):.3f}")

print("\n💡 Decision: We'll use K=5 clusters based on both methods")

## 1.4 Apply K-Means Clustering

In [ ]:
# Final K-Means with K=5
optimal_k = 5
kmeans_final = KMeans(n_clusters=optimal_k, random_state=42, n_init=10)
df['Cluster'] = kmeans_final.fit_predict(X_scaled)

print(f"✅ K-Means clustering complete with K={optimal_k}")
print(f"Final inertia: {kmeans_final.inertia_:.2f}")
print(f"Number of iterations: {kmeans_final.n_iter_}")

# Cluster distribution
print("\n📊 Cluster sizes:")
cluster_counts = df['Cluster'].value_counts().sort_index()
for cluster, count in cluster_counts.items():
    print(f"  Cluster {cluster}: {count} customers ({count/len(df)*100:.1f}%)")

## 1.5 Visualize Customer Segments

In [ ]:
# 3D visualization with Plotly
fig = plot_clusters_3d(
    X_scaled,
    df['Cluster'].values,
    centroids=kmeans_final.cluster_centers_,
    title="Customer Segments (Standardized Features)",
    feature_names=['Age (scaled)', 'Income (scaled)', 'Spending (scaled)']
)
fig.show()

print("🎨 Interactive 3D plot: Rotate, zoom, and hover to explore clusters!")

In [ ]:
# 2D scatter plots (Income vs Spending - most interesting relationship)
fig, ax = plt.subplots(figsize=(12, 8))

colors = plt.cm.Set2(np.linspace(0, 1, optimal_k))

for cluster in range(optimal_k):
    cluster_data = df[df['Cluster'] == cluster]
    ax.scatter(
        cluster_data['Annual Income (k$)'],
        cluster_data['Spending Score (1-100)'],
        s=100,
        c=[colors[cluster]],
        label=f'Cluster {cluster}',
        alpha=0.7,
        edgecolors='black',
        linewidths=0.5
    )

# Plot centroids (unscaled for interpretability)
centroids_original = scaler.inverse_transform(kmeans_final.cluster_centers_)
ax.scatter(
    centroids_original[:, 1],  # Income is 2nd feature
    centroids_original[:, 2],  # Spending is 3rd feature
    s=500,
    c='black',
    marker='X',
    edgecolors='white',
    linewidths=2,
    label='Centroids'
)

ax.set_xlabel('Annual Income (k$)', fontsize=12)
ax.set_ylabel('Spending Score (1-100)', fontsize=12)
ax.set_title('Customer Segmentation: Income vs Spending Score', fontsize=14, fontweight='bold')
ax.legend(loc='best', fontsize=10)
ax.grid(alpha=0.3)

plt.tight_layout()
plt.show()

## 1.6 Interpret Clusters and Extract Insights

In [ ]:
# Analyze cluster characteristics
cluster_summary = df.groupby('Cluster')[features].mean().round(2)
cluster_summary['Count'] = df.groupby('Cluster').size()
cluster_summary['Percentage'] = (cluster_summary['Count'] / len(df) * 100).round(1)

print("\n📋 Cluster Profiles:\n")
print(cluster_summary)

# Visualize profiles
fig, axes = plt.subplots(1, 3, figsize=(16, 5))

for idx, feature in enumerate(features):
    cluster_means = df.groupby('Cluster')[feature].mean()
    axes[idx].bar(cluster_means.index, cluster_means.values, color=colors, edgecolor='black')
    axes[idx].set_xlabel('Cluster', fontsize=11)
    axes[idx].set_ylabel(f'Average {feature}', fontsize=11)
    axes[idx].set_title(f'{feature} by Cluster', fontweight='bold')
    axes[idx].grid(alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

In [ ]:
# Name the clusters based on characteristics
cluster_names = {
    0: "Budget Shoppers",
    1: "High Rollers",
    2: "Average Joes",
    3: "Savvy Savers",
    4: "Impulse Buyers"
}

# Note: Adjust names based on actual cluster characteristics in your data
df['Segment Name'] = df['Cluster'].map(cluster_names)

print("\n🏷️ Customer Segments:\n")
for cluster in range(optimal_k):
    cluster_data = df[df['Cluster'] == cluster]
    avg_income = cluster_data['Annual Income (k$)'].mean()
    avg_spending = cluster_data['Spending Score (1-100)'].mean()
    avg_age = cluster_data['Age'].mean()
    
    print(f"Cluster {cluster}: {cluster_names.get(cluster, 'Unknown')}")
    print(f"  • Size: {len(cluster_data)} customers ({len(cluster_data)/len(df)*100:.1f}%)")
    print(f"  • Average age: {avg_age:.1f} years")
    print(f"  • Average income: ${avg_income:.1f}k")
    print(f"  • Average spending score: {avg_spending:.1f}/100")
    print()

## 1.7 Business Recommendations

Based on the customer segments, here are actionable recommendations:

In [ ]:
# Create a recommendations summary
recommendations = pd.DataFrame({
    'Cluster': list(cluster_names.keys()),
    'Segment Name': list(cluster_names.values()),
    'Marketing Strategy': [
        "Value deals, discount bundles, loyalty programs",
        "Premium products, exclusive offers, VIP experiences",
        "Balanced promotions, seasonal sales, mid-range products",
        "Quality emphasis, investment products, long-term value",
        "Impulse items near checkout, flash sales, trending products"
    ],
    'Priority': ['Medium', 'High', 'Medium', 'High', 'Medium']
})

print("\n💼 Marketing Strategy Recommendations:\n")
for idx, row in recommendations.iterrows():
    print(f"{row['Segment Name']} (Cluster {row['Cluster']}) - Priority: {row['Priority']}")
    print(f"  Strategy: {row['Marketing Strategy']}")
    print()

### Key Business Insights

1. **Target high-value segments** for premium product launches
2. **Customize email campaigns** by segment preferences
3. **Optimize store layout** based on segment shopping patterns
4. **Develop loyalty programs** tailored to each segment's behavior
5. **Allocate marketing budget** proportional to segment sizes and potential value

---

# Example 2: Image Compression

## Concept

Digital images store colors for every pixel. A 24-bit RGB image can display 16.7 million colors!

**Idea:** Use K-Means to:
1. Cluster similar colors together
2. Replace each pixel's color with its cluster centroid
3. Store only K colors instead of millions → **compression**

## Goal
Reduce image file size while maintaining visual quality.

## 2.1 Load Sample Image

In [ ]:
# Load image
image = load_sample_image()

print(f"Image shape: {image.shape}")
print(f"Dimensions: {image.shape[1]}×{image.shape[0]} pixels")
print(f"Color channels: {image.shape[2]} (RGB)")
print(f"Data type: {image.dtype}")
print(f"Value range: [{image.min()}, {image.max()}]")

In [ ]:
# Display original image
plt.figure(figsize=(8, 8))
plt.imshow(image)
plt.title('Original Image', fontsize=14, fontweight='bold')
plt.axis('off')
plt.tight_layout()
plt.show()

## 2.2 Prepare Data for Clustering

In [ ]:
# Reshape image to (n_pixels, 3) for clustering
height, width, channels = image.shape
n_pixels = height * width

# Flatten to 2D array: each row is a pixel with [R, G, B] values
pixels = image.reshape(-1, 3).astype(float)

print(f"Reshaped data: {pixels.shape}")
print(f"Each of {n_pixels:,} pixels has 3 color values (RGB)")

# Show color distribution
fig, axes = plt.subplots(1, 3, figsize=(15, 4))
colors_rgb = ['red', 'green', 'blue']
channel_names = ['Red', 'Green', 'Blue']

for i, (color, name) in enumerate(zip(colors_rgb, channel_names)):
    axes[i].hist(pixels[:, i], bins=50, color=color, alpha=0.7, edgecolor='black')
    axes[i].set_xlabel(f'{name} Channel Value (0-255)', fontsize=10)
    axes[i].set_ylabel('Frequency', fontsize=10)
    axes[i].set_title(f'{name} Channel Distribution', fontweight='bold')
    axes[i].grid(alpha=0.3)

plt.tight_layout()
plt.show()

## 2.3 Apply K-Means with Different K Values

In [ ]:
# Test different numbers of colors
k_values = [4, 8, 16, 32, 64]
compressed_images = {}

print("Compressing image with different color counts...\n")

for k in k_values:
    print(f"K={k} colors:")
    
    # Apply K-Means
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=5)
    labels = kmeans.fit_predict(pixels)
    
    # Replace each pixel with its cluster centroid
    compressed_pixels = kmeans.cluster_centers_[labels]
    
    # Reshape back to image
    compressed_image = compressed_pixels.reshape(height, width, 3).astype(np.uint8)
    compressed_images[k] = compressed_image
    
    # Calculate compression metrics
    original_colors = len(np.unique(image.reshape(-1, 3), axis=0))
    compression_ratio = (1 - k / original_colors) * 100
    
    print(f"  • Used {k} colors (reduced from {original_colors:,} unique colors)")
    print(f"  • Compression: {compression_ratio:.1f}% fewer colors stored")
    print(f"  • Inertia: {kmeans.inertia_:.2f}\n")

print("✅ Image compression complete!")

## 2.4 Visualize Compression Results

In [ ]:
# Display all versions side by side
fig, axes = plt.subplots(2, 3, figsize=(15, 10))
axes = axes.flatten()

# Original image
axes[0].imshow(image)
axes[0].set_title('Original Image\n(Full Color)', fontsize=12, fontweight='bold')
axes[0].axis('off')

# Compressed versions
for idx, (k, compressed_img) in enumerate(compressed_images.items(), 1):
    axes[idx].imshow(compressed_img)
    axes[idx].set_title(f'{k} Colors\n(K-Means Compressed)', fontsize=12, fontweight='bold')
    axes[idx].axis('off')

plt.suptitle('Image Compression with K-Means Clustering', fontsize=16, fontweight='bold', y=0.98)
plt.tight_layout()
plt.show()

print("👀 Notice how image quality changes with different numbers of colors!")

## 2.5 Quality vs Compression Trade-off

In [ ]:
# Calculate mean squared error (MSE) for each compression
mse_values = []

for k in k_values:
    compressed_img = compressed_images[k]
    mse = np.mean((image.astype(float) - compressed_img.astype(float)) ** 2)
    mse_values.append(mse)

# Plot quality metric
fig, axes = plt.subplots(1, 2, figsize=(14, 5))

# MSE vs K
axes[0].plot(k_values, mse_values, marker='o', linewidth=2, markersize=10, color='#e74c3c')
axes[0].set_xlabel('Number of Colors (K)', fontsize=12)
axes[0].set_ylabel('Mean Squared Error', fontsize=12)
axes[0].set_title('Image Quality vs Compression', fontsize=13, fontweight='bold')
axes[0].set_xscale('log', base=2)
axes[0].grid(alpha=0.3)

# Storage efficiency
original_colors = len(np.unique(image.reshape(-1, 3), axis=0))
storage_reduction = [(1 - k/original_colors) * 100 for k in k_values]

axes[1].bar(range(len(k_values)), storage_reduction, color='#2ecc71', edgecolor='black')
axes[1].set_xlabel('Compression Level', fontsize=12)
axes[1].set_ylabel('Storage Reduction (%)', fontsize=12)
axes[1].set_title('Color Palette Reduction', fontsize=13, fontweight='bold')
axes[1].set_xticks(range(len(k_values)))
axes[1].set_xticklabels([f'{k} colors' for k in k_values], rotation=45)
axes[1].grid(alpha=0.3, axis='y')

plt.tight_layout()
plt.show()

print("\n📊 Trade-off Analysis:")
for k, mse, reduction in zip(k_values, mse_values, storage_reduction):
    print(f"  K={k}: MSE={mse:.2f}, Storage reduction={reduction:.1f}%")

## 2.6 Visualize Color Palettes

In [ ]:
# Show the color palette (cluster centroids) for each K
fig, axes = plt.subplots(len(k_values), 1, figsize=(12, 8))

for idx, k in enumerate(k_values):
    # Get cluster centroids for this K
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=5)
    kmeans.fit(pixels)
    palette = kmeans.cluster_centers_.astype(np.uint8)
    
    # Create a visual palette strip
    palette_strip = np.array([palette] * 50)  # Make it tall enough to see
    
    axes[idx].imshow(palette_strip)
    axes[idx].set_title(f'Color Palette: {k} Colors', fontsize=11, fontweight='bold')
    axes[idx].set_yticks([])
    axes[idx].set_xticks(range(k))
    axes[idx].set_xticklabels(range(1, k+1), fontsize=8)

plt.suptitle('K-Means Color Palettes', fontsize=14, fontweight='bold', y=0.995)
plt.tight_layout()
plt.show()

print("🎨 Each strip shows the K representative colors chosen by K-Means")

## 2.7 Business Application

### Use Cases:

1. **Web Optimization**: Reduce image sizes for faster page loading
2. **Mobile Apps**: Save bandwidth and storage on mobile devices
3. **Archival**: Compress large image databases while maintaining visual quality
4. **Art & Design**: Extract dominant color palettes for branding
5. **Medical Imaging**: Reduce file sizes for remote diagnostics

### Recommendations:

- **K=16-32**: Good balance for web graphics (50-70% reduction with acceptable quality)
- **K=8**: Thumbnail images, icons (aggressive compression)
- **K=64**: Near-original quality for important images

### Important Note:
Modern compression (JPEG, PNG) is more sophisticated, but K-Means illustrates the core concept and can complement other techniques!

---

# Summary

In this notebook, we successfully applied K-Means to two real-world problems:

## ✅ Customer Segmentation
- Identified 5 distinct customer segments
- Analyzed characteristics (age, income, spending behavior)
- Provided actionable marketing recommendations
- **Business Value**: Targeted campaigns, personalized offers, optimized inventory

## ✅ Image Compression
- Reduced color palettes from thousands to 8-64 colors
- Achieved 50-95% color reduction
- Demonstrated quality vs compression trade-offs
- **Business Value**: Faster websites, lower bandwidth costs, efficient storage

---

## Key Takeaways

1. **Always standardize features** before applying K-Means
2. **Use multiple methods** (elbow + silhouette) to choose K
3. **Interpret results in context** - clusters have real-world meaning
4. **Visualize extensively** - 2D, 3D, and profile plots reveal insights
5. **Translate to action** - clusters are only valuable if they drive decisions

---

## What's Next?

- Try the **Streamlit app** to cluster your own data interactively
- Experiment with other datasets (time series, text, genomics)
- Compare K-Means with other clustering algorithms (DBSCAN, hierarchical)
- Explore advanced techniques (Mini-Batch K-Means for big data, constrained clustering)

---

**🎉 Congratulations!** You've completed the K-Means clustering lab. Now go apply these skills to your own projects!